# Using ASV data for Trapezoidal Integration 
*Akira Di Sandro, 6/26/20*
<br>
In this notebook, I will be trying to see if the trapezoidal integration method done on the raw data resembles the transport data Marion and her research team got. (It should match perfectly since this is the way they calculated transport, but I want to test if I can do it myself)

## 1. Load in Data

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import random

In [2]:
from scipy.io import loadmat
x = loadmat('SolomonStrait_westernXsection_linear2bot_sat4surf.mat')

## 2. Save Data into variables

In [3]:
#along strait velocity saved in 255x99x590 matrix (depth x points along cross section x timestamps)
asv = x['ASV']  #asv is in m/s

#lon: lon points along cross section, lat (same as lon), xdist: distance from start of cross section
lon = x['bathy'][0][0][2][0]
lat = x['bathy'][0][0][3][0]
xdist = x['bathy'][0][0][4][0] #xdist is in km
points = np.arange(99)

#raw_time: datenums associated to each day mooring data was taken
raw_time = x['time'][0]

#depth: array of all the depths possible
depth = x['z'][:,0]  # depth is in m

To put everything in terms of meters, we have to do the following conversions. ASV is already in m/s and depth is already in m so all we need to do is change the units of xdist from km to m.

In [4]:
xdist = xdist * (10**3)

## 3. Calculate Transport for one time stamp

### a. Just one time stamp first
I'm starting with time = 0.

In [5]:
time0 = asv[:,:,10]
time0.shape

(225, 99)

In [6]:
t0row1 = time0[0]
t0row1

array([        nan,  0.        ,  0.13979748,  0.13979748,  0.13979748,
        0.13979748,  0.13979748,  0.13979748,  0.13979748,  0.13979748,
        0.13979748,  0.13979748,  0.13979748,  0.13979748,  0.13979748,
        0.14363783,  0.14747814,  0.15131841,  0.15515864,  0.15899883,
        0.16283897,  0.16667908,  0.17051914,  0.17435916,  0.17819914,
        0.18203908,  0.18587897,  0.18971882,  0.19355863,  0.1973984 ,
        0.20123813,  0.07014786,  0.06707241,  0.06399695,  0.06092149,
        0.05784602,  0.05477056,  0.0516951 ,  0.04861963,  0.04554416,
        0.04246869,  0.03939322,  0.03631775,  0.03324228,  0.0301668 ,
        0.02709133,  0.02401585,  0.02094037,  0.01786489,  0.01478941,
        0.01171393,  0.00863844,  0.00556296,  0.00248747, -0.00058802,
       -0.00366351, -0.006739  , -0.0098145 , -0.01288999, -0.01596549,
       -0.01904099, -0.02211648, -0.02519199, -0.02826749, -0.03134299,
       -0.0344185 , -0.037494  , -0.04056951, -0.04364502, -0.04

In [7]:
nans = np.isnan(t0row1)
t0row1[nans] = 0

In [8]:
# horizontal integration of ASV
horz_int = np.trapz(t0row1, xdist)
horz_int

6944.920063968619

### b. integrate across all points in each depth

In [9]:
horz_int = []
horz_int

[]

The units for the values in horz_int are in m^2/s.

In [10]:
for row in range(time0.shape[0]):
    t0row = time0[row]
    nans = np.isnan(t0row)
    t0row[nans] = 0
    row_int = np.trapz(t0row, xdist)
    horz_int = np.append(horz_int, [row_int])

In [11]:
horz_int.shape == depth.shape

True

### c. integrate across all depths

In [12]:
transport = 0

The units for transport here is m^3/s.

In [13]:
transport = np.trapz(horz_int, depth)
transport

11393959.807636239

## 4. Make function that calculates Transport for the time-stamp you give it

I'm going to use all the code above to make a function that spits out Transport. For this calculation to work, everything has to be in the proper units (m). This function takes in four variables:
1. asv : along strait velocity, in the for of a 3D array (depth x points on cross section x time)
2. tstep : 1D array of time stamps
3. depth : 1D array of the different depths
4. dist : 1D array of the distance between point to point along the cross section
<br> 
This function returns the value of transport in Sv.

In [1]:
def calc_trans(asv, tstep, depth, dist):
    time = asv[:,:,tstep]
    horz_int = []
    for row in range(time.shape[0]):
        t0row = time[row]
        nans = np.isnan(t0row)
        t0row[nans] = 0
        row_int = np.trapz(t0row, xdist)
        horz_int = np.append(horz_int, [row_int])
    transport = np.trapz(horz_int, depth)
    transport = transport*(10**-6)
    return(transport)

In [24]:
calc_trans(asv, 10, depth, xdist)

11.393959807636238